In [2]:
import pandas as pd
import os
import pathlib

In [3]:
datafile = '..\data\schedule.xlsx'

In [5]:
vac = pd.read_excel(datafile,sheet_name='Vacation')

In [6]:
days = pd.read_excel(datafile,sheet_name='Days')

In [204]:
weeks = list(set(days.week))

In [7]:
vac_sched = days.merge(vac,on='Date')

In [8]:
#build list of all potential months call work time units (week or weekend)
#and assign this an index
potential_call = days.callshiftassignment.drop_duplicates()\
    .reset_index().drop('index',axis=1).reset_index()

In [9]:
vac_slots = vac_sched[['week','callshiftassignment','Person']].drop_duplicates()\
    .sort_values(by=['week','Person'])[['callshiftassignment','Person']].drop_duplicates()

In [10]:
vac_slots = vac_slots.merge(potential_call,on='callshiftassignment')

In [11]:
vac_slots 

,callshiftassignment,Person,index
0,weekend1,Jess,1
1,weekend8,Jess,13
2,weekday8,Jess,14
3,weekend9,Jess,15
4,weekend10,Deenah,17
5,weekday10,Deenah,18
6,weekend11,Deenah,19
7,weekend11,Erin,19
8,weekday11,Deenah,20
9,weekend12,Paul,21


In [12]:
potential_call

,index,callshiftassignment
0,0,weekday0
1,1,weekend1
2,2,weekday1
3,3,weekend2
4,4,weekday2
5,5,weekend3
6,6,weekday3
7,7,weekend4
8,8,weekday4
9,9,weekend5


In [31]:
#write the scheduled vacation weeks into a dict
#this shows which weeks/weekends people cannot work
vac_weeks = {}
for row in vac_slots.index:
    
    data = vac_slots.loc[row]
    if data[1] not in vac_weeks:
        vac_weeks[data[1]]= [data[2]]

    if data[1] in vac_weeks:
        if data[2] not in vac_weeks[data[1]]:
            vac_weeks[data[1]].append(data[2])

In [32]:
#determine which weeks/weekends people can work
#construct a dict of potential call weeks/weekends
not_vac_weeks = {}

for person in vac_weeks:
    for week in potential_call.index:

        #create first entry
        if person not in not_vac_weeks:
            if week not in vac_weeks[person]:
                not_vac_weeks[person] = [week]

        if person in not_vac_weeks:
            if week not in vac_weeks[person]:
                if week not in not_vac_weeks[person]:
                    not_vac_weeks[person].append(week)



In [42]:
not_vac_weeks['Deenah']

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 21, 22, 23]

In [18]:
import pulp

In [33]:
residents = list(set(vac.Person))

In [34]:
shifts = list(potential_call.index)

In [48]:
x = pulp.LpVariable.dicts(
    'x',
    ((shift, resident)
        for resident in residents
        for shift in shifts),
    cat=pulp.LpBinary)

In [192]:
synth_var_ix = 0
def synth_var():
    global synth_var_ix
    synth_var_ix += 1
    return synth_var_ix

def and_together(x1, x2):
    """
    produce a variable that represents x1 && x2

    That variable can then be used in constraints and the objective func.
    """
    y = pulp.LpVariable('({} AND {})_{}'.format(x1.name, x2.name, synth_var()), cat=pulp.LpBinary)
    model.addConstraint(y >= x1 + x2 - 1, 'constraint{}'.format(synth_var()))
    model.addConstraint(y <= x1, 'constraint{}'.format(synth_var()))
    model.addConstraint(y <= x2, 'constraint{}'.format(synth_var()))
    return y

def or_together(x1, x2):
    y = pulp.LpVariable('({} OR {})_{}'.format(x1.name, x2.name, synth_var()), cat=pulp.LpBinary)
    model.addConstraint(y <= x1 + x2, 'constraint{}'.format(synth_var()))
    model.addConstraint(y >= x1, 'constraint{}'.format(synth_var()))
    model.addConstraint(y >= x2, 'constraint{}'.format(synth_var()))
    return y


In [137]:
def negate(x):
    y = pulp.LpVariable('(NOT {})_{}'.format(x.name, synth_var()), cat=pulp.LpBinary)
    model.addConstraint(y == 1 - x, 'constraint_{}'.format(synth_var()))
    return y

In [138]:
def or_all(xs):
    if len(xs) == 2:
        return or_together(*xs)
    if len(xs) == 1:
        return xs[0]
    if len(xs) == 0:
        raise ValueError('Cannot OR together zero variables')
    x1, x2, *x_rest = xs
    y = or_together(x1, x2)
    return or_all([*x_rest, y])


In [189]:
def avg(xs):
    return sum(xs) / len(xs)

In [207]:
def no_two_shifts_in_a_row(resident):
    sequential_shifts = []
    for m1, m2 in zip(shifts, shifts[1:]):
        sequential_shifts.append(
            negate([x[shift, resident] for shift in shifts]),
                or_all([x[shift, resident] for shift in shifts]),
        )
    # avg rather than sum below because each of "NOT (Jan inpatient & Feb inpatient)" is a request
    return avg(sequential_shifts)

In [243]:
model = pulp.LpProblem('Schedule', pulp.LpMaximize)

#constraints for residents
for resident in residents:

    # this is the "no time-turners constraint"
     
    #minumum number of shifts 
    model.addConstraint(
        sum(x[shift, resident] for shift in shifts) >= 5,
        F'{6} shifts per resident {resident}')

    #remove vacation weeks
    for s in vac_weeks[resident]:
        model.addConstraint(
                sum(x[s, resident] for shift in shifts) == 0 ,
                '{} cannot work during {}'.format(resident,s))

    #no back to back call
    #no_two_shifts_in_a_row(resident)
    #remove vacation weeks
    for s in not_vac_weeks[resident]:
        for i in not_vac_weeks[resident][1:]:
            if i - s == 1:
                model.addConstraint(
                        sum(x[s, resident] for shift in shifts)
                        + sum(x[i, resident] for shift in shifts) <= len(shifts) ,
                        '{} cannot work during {}'.format(resident,s))
    
    

    



for shift in shifts:
    
    #no duplication of shifts
    model.addConstraint(
        sum(x[shift, resident] for resident in residents) == 1,
        F'{1} resident per {shift}')



            #model.addConstraint(
            #        sum(x[minus, resident] for resident in residents)+ sum(x[s, resident] for resident in residents) == 0 ,
            #        '{} cannot work shift {} before {}'.format(resident,s,minus))


In [244]:
# pulp.LpSolverDefault.msg = 1
model.solve()
if pulp.LpStatus[model.status] != 'Optimal':
    raise ValueError(pulp.LpStatus[model.status])

data = [{
    'resident': resident,
    'shift':shift,
    'result': x[shift,resident].varValue}
 for resident in residents for shift in shifts]

result_raw = pd.DataFrame(data)
result_final = result_raw[result_raw.result > 0]
result_final

ValueError: Infeasible

In [240]:
result_final[result_final.resident == 'Deenah']

,resident,shift,result
25,Deenah,1,1.0
31,Deenah,7,1.0
33,Deenah,9,1.0
36,Deenah,12,1.0
38,Deenah,14,1.0
40,Deenah,16,1.0
45,Deenah,21,1.0
47,Deenah,23,1.0


In [241]:
result_final_out = result_final.merge(potential_call,left_on='shift',right_on='index')\
    .merge(days,on='callshiftassignment')

In [242]:
result_final_out.to_excel('..\data\cl_planned_schedule.xlsx',sheet_name='cl_shifts',index=False)